In [1]:
!pip install --upgrade ngboost --quiet

     |████████████████████████████████| 348 kB 35.6 MB/s 
     |████████████████████████████████| 55 kB 4.2 MB/s 


In [90]:
from ngboost import NGBRegressor

from sklearn.datasets import load_boston, load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Lasso

In [98]:
class Config:
    SEED = 42
    N_ESTIMATORS = 5_000
    VERBOSE_EVAL = 50
    EARLY_STOPPOING_ROUNDS = 100

In [99]:
X, y = load_boston(True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, test_size=0.2, random_state=Config.SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=.8, test_size=0.2, random_state=Config.SEED)

print(X_train.shape[0], X_valid.shape[0], X_test.shape[0])

323 81 102


In [101]:
base_learner = DecisionTreeRegressor(
    criterion='friedman_mse', 
    max_depth=3, 
    ccp_alpha=.05
    )

ngb = NGBRegressor(
    Base=base_learner,
    n_estimators=Config.N_ESTIMATORS, 
    learning_rate=1e-3,
    minibatch_frac=0.5, 
    col_sample=1,
    verbose_eval=Config.VERBOSE_EVAL, 
    random_state=Config.SEED
    )

ngb.fit(
    X=X_train, 
    Y=y_train,
    X_val=X_valid, 
    Y_val=y_valid, 
    early_stopping_rounds=Config.EARLY_STOPPOING_ROUNDS
    )

[iter 0] loss=3.6947 val_loss=3.5637 scale=1.0000 norm=7.1854
[iter 50] loss=3.6085 val_loss=3.5238 scale=1.0000 norm=6.9416
[iter 100] loss=3.5594 val_loss=3.4824 scale=1.0000 norm=6.7920
[iter 150] loss=3.4518 val_loss=3.4357 scale=2.0000 norm=11.6379
[iter 200] loss=3.4064 val_loss=3.3925 scale=2.0000 norm=11.2172
[iter 250] loss=3.3595 val_loss=3.3530 scale=2.0000 norm=10.6290
[iter 300] loss=3.2966 val_loss=3.3141 scale=2.0000 norm=9.4701
[iter 350] loss=3.2825 val_loss=3.2758 scale=2.0000 norm=9.5276
[iter 400] loss=3.2368 val_loss=3.2378 scale=2.0000 norm=8.7284
[iter 450] loss=3.2018 val_loss=3.2008 scale=2.0000 norm=8.5428
[iter 500] loss=3.1396 val_loss=3.1648 scale=2.0000 norm=7.4269
[iter 550] loss=3.1169 val_loss=3.1286 scale=2.0000 norm=7.5834
[iter 600] loss=3.0429 val_loss=3.0944 scale=2.0000 norm=6.4336
[iter 650] loss=3.0096 val_loss=3.0615 scale=2.0000 norm=6.1540
[iter 700] loss=2.9673 val_loss=3.0275 scale=2.0000 norm=6.0168
[iter 750] loss=2.9461 val_loss=2.9951 s

NGBRegressor(Base=DecisionTreeRegressor(ccp_alpha=0.05,
                                        criterion='friedman_mse', max_depth=3,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort='deprecated', random_state=None,
                                        splitter='best'),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.LogScore'>, col_sample=1,
             learning_rate=0.001, minibatch_frac=0.5, n_estimators=5000,
             natural_gradient=True,
             random_state=RandomState(MT19937) at 0x7F7CA0AA3E20, tol=0.0001,
             verbose=True, verbose_eval=50)

In [102]:
y_pred = ngb.predict(X_valid, max_iter=ngb.best_val_loss_itr)
y_dist = ngb.pred_dist(X_valid, max_iter=ngb.best_val_loss_itr)

# valid Root Mean Squared Error
valid_RMSE = mean_squared_error(y_valid, y_pred, squared=False)
print('Valid MSE', valid_RMSE)

# valid Negative Log Likelihood
valid_NLL = -y_dist.logpdf(y_valid).mean()
print('Valid NLL', valid_NLL)

Valid MSE 3.243297970476864
Valid NLL 2.58443182298427


In [103]:
y_pred = ngb.predict(X_test, max_iter=ngb.best_val_loss_itr)
y_dist = ngb.pred_dist(X_test, max_iter=ngb.best_val_loss_itr)

# valid Root Mean Squared Error
test_RMSE = mean_squared_error(y_test, y_pred, squared=False)
print('Test MSE', test_RMSE)

# valid Negative Log Likelihood
test_NLL = -y_dist.logpdf(y_test).mean()
print('Test NLL', test_NLL)

Test MSE 3.1936898638959677
Test NLL 2.545081730838024


In [97]:
## Feature importance for loc trees
feature_importance_loc = ngb.feature_importances_[0]

## Feature importance for scale trees
feature_importance_scale = ngb.feature_importances_[1]

df_loc = pd.DataFrame({'feature':load_boston()['feature_names'],
                       'importance':feature_importance_loc})\
    .sort_values('importance', ascending=False)
df_scale = pd.DataFrame({'feature':load_boston()['feature_names'],
                       'importance':feature_importance_scale})\
    .sort_values('importance', ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(13,6))
fig.suptitle("Feature importance plot for distribution parameters", fontsize=17)
sns.barplot(x='importance',y='feature',ax=ax1,data=df_loc, color="skyblue").set_title('loc param')
sns.barplot(x='importance',y='feature',ax=ax2,data=df_scale, color="skyblue").set_title('scale param')

ValueError: ignored